In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser

In [ ]:
# smoothing
def smoothFilter(col, window_size):
    ones = np.ones(int(window_size))/float(window_size)
    return np.convolve(col, ones, 'same')

# smoothFilter(cb_ship.speed, 5)
# plt.figure(figsize=(20,3))
# plt.plot(range(len(cb_ship.speed)), cb_ship.speed)
# plt.plot(range(len(smoothFilter(cb_ship.speed, 7))), smoothFilter(cb_ship.speed, 7))

# plt.show()

In [ ]:
# import data
cb_ship = pd.read_csv('data.csv')

# Remove nan and fill missing values
cols = ['speed', 'VMDR', 'VHM0', 'Temperature_surface']
cb_ship[cols].apply(pd.to_numeric, errors='coerce')
cb_ship.sort_values(by=['Date/Time (UTC)'], inplace=True)
cb_ship = cb_ship.dropna(subset=cols)[::10]
utc = list(cb_ship['Date/Time (UTC)'])
fdate = [parser.parse(t) for t in utc]

# extract lat/lon 
lat = cb_ship['Latitude']
lon = cb_ship['Longitude']
coord = np.column_stack((list(lon),list(lat)))
y, x = coord[:,0], coord[:,1]

# temperature
temperature = np.array(cb_ship['Temperature_surface']) - 273.15

# wave hight and direction
waveDir = list(cb_ship['VMDR'])
waveH = np.array(cb_ship['VHM0'])

# vessel speed
speed = list(smoothFilter(cb_ship.speed, 5))

eng_col = ['Load E1', 'Load E2', 'Load E3', 'Load E4']
df_engin = cb_ship[eng_col]

# create a vector using 'eastward_wind' and 'northward_wind'
windSpeed = np.sqrt((cb_ship['eastward_wind']*cb_ship['eastward_wind']) + (cb_ship['northward_wind']*cb_ship['northward_wind'])).values
windDir = list(np.degrees(np.arctan(cb_ship['northward_wind']/cb_ship['eastward_wind'])))

In [ ]:

# create plots 
fig = make_subplots(
    rows=6, cols=3,
    specs=[
            [{"type": "scattergeo", "rowspan": 6}, {"type": "polar", "rowspan": 3},{"type": "polar", "rowspan": 3}],
            [None,None,None],
            [None,None,None],
            [None,{"type": "xy", "colspan": 2}, None],
            [None,{"type": "scatter", "colspan": 2}, None],
            [None,{"type": "scatter", "colspan": 2}, None]])

# map with route 
fig.add_trace(go.Scattergeo(lat=x, lon= y, mode='lines', name='track', line = dict(width=3, color='blue')),
    row=1, col=1
)
fig.add_trace(go.Scattergeo(lat=[x[0]], lon=[y[0]], name='Position', mode='markers', marker=dict(color='red')),
    row=1, col=1
)


fig.add_trace(go.Barpolar(
   r=[waveH[0]],
    theta=[waveDir[0]],
    width=[4],
    marker_line_width=0, 
    name='Wave sign. height (m) and direction °'),
    row=1, col=2
)

fig.add_trace(go.Barpolar(
    r=[windSpeed[0]],
    theta=[windDir[0]],
    width=[4],
    marker_line_width=0, 
    name='Wind speed (m/s) and direction  °'),
    row=1, col=3
)

# engins 
fig.add_trace(go.Scatter(x=[fdate[0]], y=[list(df_engin['Load E1'])[0]], mode='markers', showlegend=False, marker=dict(color='red')),
    row=4, col=2
)

fig.add_trace(go.Scatter(x=[fdate[0]], y=[list(df_engin['Load E2'])[0]], mode='markers', showlegend=False, marker=dict(color='red')),
    row=4, col=2
)
fig.add_trace(go.Scatter(x=[fdate[0]], y=[list(df_engin['Load E3'])[0]], mode='markers', showlegend=False, marker=dict(color='red')),
    row=4, col=2
)
fig.add_trace(go.Scatter(x=[fdate[0]], y=[list(df_engin['Load E4'])[0]], mode='markers', showlegend=False, marker=dict(color='red')),
    row=4, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=df_engin['Load E1'], mode='lines', name='Load E1'),
    row=4, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=df_engin['Load E2'], mode='lines', name='Load E2'),
    row=4, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=df_engin['Load E3'], mode='lines', name='Load E3'),
    row=4, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=df_engin['Load E4'], mode='lines', name='Load E4'),
    row=4, col=2
)
# Temperature
fig.add_trace(go.Scatter(x=[fdate[0]], y=[temperature[0]], showlegend=False, mode='markers', marker=dict(color='red')),
    row=5, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=temperature, mode='lines', name='Temperature'),
    row=5, col=2
)


# speed height
fig.add_trace(go.Scatter(x=[fdate[0]], y=[speed[0]], showlegend=False, mode='markers', marker=dict(color='red')),
  row=6, col=2
)


fig.add_trace(go.Scatter(x=fdate, y=speed, mode='lines', name="speed"),
    row=6, col=2
)


# create sequence of frames 
fig.frames=[go.Frame(
        data=[
            # update map dot
            go.Scattergeo(lat=x,lon=y, mode='lines', line = dict(width=3, color='blue')),
            go.Scattergeo(
            lat=[x[k]],
            lon=[y[k]], mode='markers', marker=dict(color='red')),

            # update wind force 
            go.Barpolar(
                    r=[waveH[k]],
                    theta=[waveDir[k]],
                    width=[4],
                    marker_line_width=0
                ),
            
            # update wave hight 
           go.Barpolar(
                r=[windSpeed[k]],
                theta=[windDir[k]],
                width=[4],
                marker_line_width=0
            ),

            # update engins
            go.Scatter(x=[fdate[k]], y=[list(df_engin['Load E1'])[k]], mode='markers', showlegend=False, marker=dict(color='red')),
            go.Scatter(x=[fdate[k]], y=[list(df_engin['Load E2'])[k]], mode='markers', showlegend=False, marker=dict(color='red')),
            go.Scatter(x=[fdate[k]], y=[list(df_engin['Load E3'])[k]], mode='markers', showlegend=False, marker=dict(color='red')),
            go.Scatter(x=[fdate[k]], y=[list(df_engin['Load E4'])[k]], mode='markers', showlegend=False, marker=dict(color='red')),
            go.Scatter(x=fdate, y=df_engin['Load E1'], mode='lines', name='Load E1'),
            go.Scatter(x=fdate, y=df_engin['Load E2'], mode='lines', name='Load E2'),
            go.Scatter(x=fdate, y=df_engin['Load E3'], mode='lines', name='Load E3'),
            go.Scatter(x=fdate, y=df_engin['Load E4'], mode='lines', name='Load E4'),
            
            # update temperature 
            go.Scatter(x=[fdate[k]],y=[temperature[k]], mode='markers', marker=dict(color='red')),
            go.Scatter(x=fdate, y=temperature, mode='lines'),
            
            # update speed
            go.Scatter(x=[fdate[k]],y=[speed[k]], mode='markers', marker=dict(color='red')),
            go.Scatter(x=fdate, y=speed, mode='lines'),
            
          
        ],  name=str(k))
        for k in range(coord.shape[0])]



sliders = [
            {
                "steps": [
                   {
                        "args": [[f.name], { "frame": {"duration": 0}, "mode": "update", "redraw": False}],
                        "label": fdate[k].strftime("%d.%m.%Y"),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

fig.update_yaxes(title_text='Percent %', showticklabels=False, row=4, col=2, range=[fdate[0],fdate[-1]])
fig.update_yaxes(title_text='C°', showticklabels=False, row=5, col=2, range=[fdate[0],fdate[-1]])
fig.update_yaxes(title_text='m/s', showticklabels=False, row=6, col=2, range=[fdate[0],fdate[-1]])
# fig.update_xaxes(showticklabels=False, row=7, col=2, range=[fdate[0],fdate[-1]])

   # ['equirectangular', 'mercator', 'orthographic', 'natural
   #          earth', 'kavrayskiy7', 'miller', 'robinson', 'eckert4',
   #          'azimuthal equal area', 'azimuthal equidistant', 'conic
   #          equal area', 'conic conformal', 'conic equidistant',
   #          'gnomonic', 'stereographic', 'mollweide', 'hammer',
   #          'transverse mercator', 'albers usa', 'winkel tripel',
   #          'aitoff', 'sinusoidal']
fig.update_geos(projection_type="orthographic")
fig.update_layout( autosize=False, template='plotly_dark',
        width=1800,
        height=900,
        title_text="Scrubber data with environmental data", 
        hovermode="closest",
        sliders=sliders,
        polar = dict(radialaxis = dict(range=[0,max(waveH)], title='m'), angularaxis = dict(direction='clockwise', rotation = 90)),
        polar2 = dict(radialaxis = dict(range=[0,max(windSpeed)], title='m/s'), angularaxis = dict(direction='clockwise',  rotation = 90))
        )

# fig.show()

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)